### Loading the model

In [2]:
import sys
sys.path.insert(0, '../')

from transformers import BertConfig
from model.model import MidiBert
import pickle
import torch
import numpy as np


print("Loading Dictionary")
with open('/root/code/HugoA45/music_project/music_project/model/CP.pkl', 'rb') as f:
    e2w, w2e = pickle.load(f)

# Define the configuration for the BERT model
configuration = BertConfig(
    max_position_embeddings=512,
    position_embedding_type='relative_key_query',
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=12
)

# Initialize the model
midibert = MidiBert(bertConfig=configuration, e2w=e2w, w2e=w2e)

# Define the path to your checkpoint here
ckpt_path = '/root/code/HugoA45/music_project/music_project/model/pretrain_model.ckpt'

# Load the checkpoint
checkpoint = torch.load(ckpt_path, map_location='cpu')

# Remove the unexpected key from the state dictionary
if "bert.embeddings.position_ids" in checkpoint['state_dict']:
    embeddings_position_ids =  checkpoint['state_dict']["bert.embeddings.position_ids"]
    del checkpoint['state_dict']["bert.embeddings.position_ids"]

# Load the state dictionary from the checkpoint into the model
midibert.load_state_dict(checkpoint['state_dict'])
midibert.eval()


/root/.pyenv/versions/3.10.6/envs/proj_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading Dictionary


MidiBert(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
              (distance_embedding): Embedding(1023, 64)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Layer

In [3]:
data = np.load('/root/code/HugoA45/music_project/music_project/data/emopia/emopia_test.npy')
data_ans = np.load('/root/code/HugoA45/music_project/music_project/data/emopia/emopia_test_ans.npy')


In [4]:
data[0:10,:,:].shape


(10, 512, 4)

In [5]:
data_ans


array([0, 3, 0, 0, 3, 1, 3, 3, 1, 2, 1, 3, 2, 2, 1, 0, 0, 1, 2, 2, 3, 2,
       3, 3, 1, 3, 3, 1, 1, 0, 3, 2, 1, 2, 0, 2, 0, 3, 1, 0, 3, 3, 3, 0,
       1, 2, 3, 1, 1, 1, 1, 3, 3, 3, 3, 3, 1, 1, 2, 2, 0, 2, 3, 1, 2, 2,
       2, 1, 1, 0, 3, 0, 0, 2, 0, 0, 2, 0, 2, 2, 3, 2, 0, 1, 0, 3, 1, 0,
       0, 3, 2, 2, 3, 0, 1, 3, 0, 1, 3, 0, 2, 0, 1, 3, 1, 3, 0, 2, 1])

In [6]:
midibert.bert


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (distance_embedding): Embedding(1023, 64)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 

In [7]:
res = midibert.forward(torch.from_numpy(data[0:5,0:512,:]))


In [8]:
res.last_hidden_state.shape
# (batch_size, sequence_length, hidden_size)
#
#‘batch_size’ is the number of inputs you have,
#‘sequence_length’ is the length of your input sequences, and
#‘hidden_size’ is the size of the hidden layers in the BERT model.


torch.Size([5, 512, 768])

In [9]:
composer_names = ['M', 'C', 'E','H','W','J','S','Y']
composer_names = np.arange(0,8,1)


In [10]:
import torch
import torch.nn.functional as F


midibert.classifier = torch.nn.Linear(configuration.hidden_size, len(composer_names))


logits = midibert.classifier(res.last_hidden_state)


probs = F.softmax(logits, dim=-1)
avg_probs = torch.mean(probs, dim=1)
_, predicted_class = torch.max(avg_probs, 1)
predicted_composers = [composer_names[idx.item()] for idx in predicted_class]


In [11]:
predicted_composers, data_ans


([2, 2, 2, 2, 2],
 array([0, 3, 0, 0, 3, 1, 3, 3, 1, 2, 1, 3, 2, 2, 1, 0, 0, 1, 2, 2, 3, 2,
        3, 3, 1, 3, 3, 1, 1, 0, 3, 2, 1, 2, 0, 2, 0, 3, 1, 0, 3, 3, 3, 0,
        1, 2, 3, 1, 1, 1, 1, 3, 3, 3, 3, 3, 1, 1, 2, 2, 0, 2, 3, 1, 2, 2,
        2, 1, 1, 0, 3, 0, 0, 2, 0, 0, 2, 0, 2, 2, 3, 2, 0, 1, 0, 3, 1, 0,
        0, 3, 2, 2, 3, 0, 1, 3, 0, 1, 3, 0, 2, 0, 1, 3, 1, 3, 0, 2, 1]))

### Bert model

In [12]:
midibert.bert


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
            (distance_embedding): Embedding(1023, 64)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 

### Number of parameters

In [17]:
model = midibert.bert.from_pretrained('bert-base-uncased')
num_parameters = sum(p.numel() for p in model.parameters())

print(f'The model has {num_parameters} parameters.')


The model has 109482240 parameters.


 In a typical BERT model, the state_dict keys for the transformer layers usually follow this pattern:

bert.encoder.layer.{i}.{sub_layer}.{param}
where:

{i} is the layer index (from 0 to 11 for a 12-layer BERT model),
{sub_layer} is the sub-layer type, which can be attention.self, attention.output, intermediate, or output,
{param} is the parameter type, which can be weight or bias for linear layers, and gamma or beta for layer normalization.
To count the number of unique transformer layers in the state_dict, you can extract the layer index from each key and add it to a set (which automatically removes duplicates), then count the number of elements in the set.

In [18]:
state_dict = checkpoint['state_dict']

# Initialize an empty set to store the layer indices
layer_indices = set()

# Iterate over the keys in the state dictionary
for key in state_dict.keys():
    # Split the key into parts
    parts = key.split('.')
    # If this key corresponds to a transformer layer
    if parts[0] == 'bert' and parts[1] == 'encoder' and parts[2] == 'layer':
        # Extract the layer index and add it to the set
        layer_indices.add(int(parts[3]))

# Print the number of unique layers
print(f"The model has {len(layer_indices)} unique transformer layers.")


The model has 12 unique transformer layers.


In [19]:
# Get the names of the layers
layer_names = state_dict.keys()

# Get the unique keys from the layers
unique_keys = set()
for name in layer_names:
    # Split the name by '.' and get the first part
    key = name.split('.')[0]
    unique_keys.add(key)

# Print the unique keys
for key in unique_keys:
    print(key)


word_emb
in_linear
bert


the state_dict keys for the transformer layers usually follow this pattern:

bert.encoder.layer.{i}.{sub_layer}.{param}
where:

{i} is the layer index (from 0 to 11 for a 12-layer BERT model),
{sub_layer} is the sub-layer type, which can be attention.self, attention.output, intermediate, or output,
{param} is the parameter type, which can be weight or bias for linear layers, and gamma or beta for layer normalization.
To count the number of unique transformer layers in the state_dict, you can extract the layer index from each key and add it to a set (which automatically removes duplicates), then count the number of elements in the set.

e2w

In [20]:
e2w.keys()


dict_keys(['Bar', 'Position', 'Pitch', 'Duration'])

In [21]:
w2e['Bar'], w2e['Position'], w2e['Pitch'], w2e['Duration']


({0: 'Bar New', 1: 'Bar Continue', 2: 'Bar <PAD>', 3: 'Bar <MASK>'},
 {0: 'Position 1/16',
  1: 'Position 2/16',
  2: 'Position 3/16',
  3: 'Position 4/16',
  4: 'Position 5/16',
  5: 'Position 6/16',
  6: 'Position 7/16',
  7: 'Position 8/16',
  8: 'Position 9/16',
  9: 'Position 10/16',
  10: 'Position 11/16',
  11: 'Position 12/16',
  12: 'Position 13/16',
  13: 'Position 14/16',
  14: 'Position 15/16',
  15: 'Position 16/16',
  16: 'Position <PAD>',
  17: 'Position <MASK>'},
 {0: 'Pitch 22',
  1: 'Pitch 23',
  2: 'Pitch 24',
  3: 'Pitch 25',
  4: 'Pitch 26',
  5: 'Pitch 27',
  6: 'Pitch 28',
  7: 'Pitch 29',
  8: 'Pitch 30',
  9: 'Pitch 31',
  10: 'Pitch 32',
  11: 'Pitch 33',
  12: 'Pitch 34',
  13: 'Pitch 35',
  14: 'Pitch 36',
  15: 'Pitch 37',
  16: 'Pitch 38',
  17: 'Pitch 39',
  18: 'Pitch 40',
  19: 'Pitch 41',
  20: 'Pitch 42',
  21: 'Pitch 43',
  22: 'Pitch 44',
  23: 'Pitch 45',
  24: 'Pitch 46',
  25: 'Pitch 47',
  26: 'Pitch 48',
  27: 'Pitch 49',
  28: 'Pitch 50',
  29

1. Preprocessing the Input Data
Tokenization: Convert your input text into tokens. BERT uses a specific tokenizer that breaks text into tokens understood by the model.
Adding Special Tokens: BERT requires special tokens at the beginning and end of each sequence (e.g., [CLS] at the start and [SEP] at the end).
Padding and Truncation: Sequences should be of the same length. You may need to pad shorter sequences with a special token (e.g., [PAD]) or truncate longer ones.
Creating Attention Masks: Attention masks tell the model which tokens should be paid attention to and which are just padding.
2. Loading the Pretrained BERT Model
Select the Appropriate Model: Depending on your task (e.g., text classification, question answering), choose a BERT model pretrained for that task.
Load the Model: Use a deep learning framework like PyTorch or TensorFlow to load the model. These frameworks often have APIs or libraries (like Hugging Face's Transformers) that simplify this process.
3. Making a Prediction
Input Preparation: Convert your preprocessed input data into a format compatible with your deep learning framework, typically tensors.
Model Inference: Pass your input through the model. The model will output logits, which are raw prediction values.
Interpreting Output: For classification tasks, apply a softmax function to the logits to get probabilities. For other tasks, the interpretation will vary.
4. Postprocessing the Output
Extract Predictions: Based on the task, extract the required information from the output (e.g., the class with the highest probability for classification tasks).
Handling Special Cases: For more complex tasks like question answering, you may need additional postprocessing to extract the answer from the output tokens.